# Notebook 02 | Agregações Gold Mensal

**Objetivos:**

- Carregar a camada Silver com vendas em USD  
- Agregar mensalmente `total_sales_usd` de todas as `storekey`  
- Gerar série temporal mensal contínua para modelagem ARIMA  
- Gravar a camada Gold como Parquet pronta para análise  


## 1. Configuração do Ambiente

Importar funções do Spark e definir variáveis de caminho no DBFS.


In [0]:
# Importar funções do Spark SQL para manipulação de data e agregações
from pyspark.sql.functions import (
    col, to_date, trunc, sum as _sum,
    min as _min, max as _max, sequence, explode, expr
)

# Definir nomes de banco e caminhos no DBFS
silver_db   = "silver"
gold_db     = "gold"
gold_path   = "/FileStore/gold/sales_monthly_gold"

## 2. Carregamento da Camada Silver

Ler a tabela `sales_silver` registrada no catálogo `silver`.


In [0]:
# Carregar DataFrame da camada Silver a partir do catálogo
df_silver = spark.table(f"{silver_db}.sales_silver")

## 3. Agregação Mensal

Criar coluna de início de mês e somar `total_sales_usd` por mês.

In [0]:
# Garantir que 'order_date' está no tipo date
df_silver = df_silver.withColumn("order_date", to_date(col("order_date")))

# Agregar vendas por início de mês
df_monthly = (
    df_silver
      .withColumn("month_start", trunc(col("order_date"), "month"))
      .groupBy("month_start")
      .agg(_sum("total_sales_usd").alias("total_sales_usd"))
)

## 4. Geração de Série Temporal Mensal Contínua

Construir sequência de todos os meses entre o mínimo e o máximo e preencher meses sem vendas com zero.

In [0]:
# Determinar datas mínima e máxima na agregação mensal
bounds_df = df_monthly.agg(
    _min("month_start").alias("min_date"),
    _max("month_start").alias("max_date")
)

# Gerar sequência de datas mensais contínua
month_seq_df = (
    bounds_df
      .select(sequence(col("min_date"), col("max_date"), expr("interval 1 month")).alias("month_seq"))
      .select(explode(col("month_seq")).alias("month_start"))
)

# Unir sequência ao DF mensal e preencher zeros onde não havia vendas
df_gold = (
    month_seq_df
      .join(df_monthly, on="month_start", how="left")
      .fillna({"total_sales_usd": 0})
)

## 5. Escrita da Camada Gold

Gravar como Parquet no DBFS e registrar a tabela no catálogo `gold`.


In [0]:
# Gravar DataFrame Gold em formato Parquet
df_gold.write \
    .format("parquet") \
    .mode("overwrite") \
    .save(f"{gold_path}")

# Criar banco e tabela no catálogo para facilitar consultas
spark.sql(f"CREATE DATABASE IF NOT EXISTS {gold_db}")
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {gold_db}.sales_monthly_gold
    USING PARQUET
    LOCATION '{gold_path}'
""")


Out[13]: DataFrame[]

## 6. Verificação Final

Mostrar esquema e algumas linhas da tabela Gold para checagem.


In [0]:
# Exibir esquema e primeiras linhas do DataFrame Gold
df_gold.printSchema()
df_gold.show(10, False)



root
 |-- month_start: date (nullable = false)
 |-- total_sales_usd: double (nullable = false)

+-----------+------------------+
|month_start|total_sales_usd   |
+-----------+------------------+
|2016-01-01 |645982.101124     |
|2016-02-01 |885621.1520109985 |
|2016-03-01 |332996.9424040003 |
|2016-04-01 |111698.406021     |
|2016-05-01 |566862.1989969998 |
|2016-06-01 |502270.70808399975|
|2016-07-01 |437780.70451899973|
|2016-08-01 |534085.3412010001 |
|2016-09-01 |581498.5341419992 |
|2016-10-01 |609006.9330939996 |
+-----------+------------------+
only showing top 10 rows



## 7. Próximos Passos

- Explorar visualmente a série temporal agregada de vendas  
- Identificar padrões, sazonalidades e outliers para tratamento pré-modelagem  
- Iniciar a aplicação de modelos preditivos (SARIMA, Prophet) com janelas definidas  
